# import 

In [9]:
import pandas as pd
import os
import numpy as np
import gc

# Auto Guon

In [ ]:
!pip install -q autogluon

In [ ]:
!pip install --upgrade scikit-learn
!pip install --upgrade autogluon

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
data_root = '/content/'
train_data = TabularDataset(data_root + 'train.csv')
test_data = TabularDataset(data_root + 'test.csv')

predictor = TabularPredictor(label='target').fit(train_data=train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240523_071328"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240523_071328"
=================== System

In [4]:
predictions = predictor.predict(test_data)
predictions.value_counts()

target
0    20000
Name: count, dtype: int64

In [5]:
predictions_prob = predictor.predict_proba(test_data)

In [6]:
predictions_prob

,0,1
0,0.971455,0.028545
1,0.971345,0.028655
2,0.861287,0.138714
3,0.813649,0.186351
4,0.962809,0.037191
...,...,...
19995,0.876360,0.123640
19996,0.963024,0.036975
19997,0.945385,0.054615
19998,0.970163,0.029837


In [7]:
predictions_prob.iloc[:,1]

0        0.028545
1        0.028655
2        0.138714
3        0.186351
4        0.037191
           ...   
19995    0.123640
19996    0.036975
19997    0.054615
19998    0.029837
19999    0.012090
Name: 1, Length: 20000, dtype: float64

In [22]:
feature_importance = predictor.feature_importance(train_data)
feature_importance

These features in provided data are not utilized by the predictor and will be ignored: ['deferredmnthsnum_166L', 'formonth_206L', 'forquarter_1017L', 'forquarter_462L', 'fortoday_1092L', 'forweek_1077L', 'forweek_601L', 'foryear_818L', 'mastercontrexist_109L', 'max_totaloutstanddebtvalue_668A', 'max_contractsum_5085717L', 'max_numberofoutstandinstls_520L']
Computing feature importance via permutation shuffling for 297 features using 5000 rows with 5 shuffle sets...
	1412.16s	= Expected runtime (282.43s per shuffle set)
	214.92s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
avgdpdtolclosure24_3658938P,1.680000e-03,0.000460,0.000614,5,0.002628,0.000732
pctinstlsallpaidlate6d_3546844L,1.680000e-03,0.000390,0.000324,5,0.002483,0.000877
maxdpdlast24m_143P,1.640000e-03,0.000385,0.000338,5,0.002432,0.000848
pctinstlsallpaidlate4d_3546849L,1.560000e-03,0.000385,0.000410,5,0.002352,0.000768
pctinstlsallpaidlat10d_839L,1.520000e-03,0.000303,0.000181,5,0.002145,0.000895
...,...,...,...,...,...,...
max_dpdmaxdateyear_896T,0.000000e+00,0.000000,0.500000,5,0.000000,0.000000
max_totalamount_996A,-2.220446e-17,0.000141,0.500000,5,0.000291,-0.000291
max_dpdmaxdatemonth_89T,-4.000000e-05,0.000089,0.813050,5,0.000144,-0.000224
maxannuity_4075009A,-4.000000e-05,0.000089,0.813050,5,0.000144,-0.000224


In [23]:
feature_importance.to_csv('feature_importance.csv', index=False)

# Submit

In [11]:
sampla = pd.read_parquet('/content/test.parquet')
sampla

,case_id,date_decision,MONTH
0,14256,2021-05-01,202105.0
1,1348,2020-11-20,202011.0
2,13475,2021-05-01,202105.0
3,1120,2020-11-22,202011.0
4,11878,2021-05-04,202105.0
...,...,...,...
19995,3444,2020-11-20,202011.0
19996,19016,2021-04-30,202104.0
19997,4171,2020-11-26,202011.0
19998,10063,2021-04-30,202104.0


In [12]:
auto_gluon_pred = pd.DataFrame()
auto_gluon_pred['case_id'] = sampla['case_id']
auto_gluon_pred['target'] = ''
auto_gluon_pred['target'] = predictions_prob.iloc[:,1]
auto_gluon_pred

,case_id,target
0,14256,0.028545
1,1348,0.028655
2,13475,0.138714
3,1120,0.186351
4,11878,0.037191
...,...,...
19995,3444,0.123640
19996,19016,0.036975
19997,4171,0.054615
19998,10063,0.029837


In [14]:
sample_submition = pd.read_csv('/content/sample_submission.csv')
sample_submition.drop(columns='target', inplace=True)
sample_submition

,case_id
0,16791
1,12423
2,19352
3,17099
4,7491
...,...
19995,1268
19996,17450
19997,15421
19998,10109


In [15]:
merged_df = pd.merge(sample_submition, auto_gluon_pred, on='case_id', how='left')
merged_df

,case_id,target
0,16791,0.024528
1,12423,0.107759
2,19352,0.009219
3,17099,0.039340
4,7491,0.112381
...,...,...
19995,1268,0.061662
19996,17450,0.008208
19997,15421,0.023188
19998,10109,0.029778


In [16]:
merged_df.to_csv('submission.csv', index=False)